## **Arc to Cubic Bezier Curve**
The functions are taken from the following link: https://github.com/Shriinivas/arc-to-curve-python-conversion/blob/master/a2c.py

The code is responsible for translating the arcs into cubic bezier curves. An arc because it can be a circle or 2π angle, it translates at maximum 4 cubic bezier because it is π/2.

In [1]:
import math
### Python conversion of the javascript a2c function
### Original function at: https://github.com/fontello/svgpath

# Convert an arc to a sequence of cubic bezier curves
#

TAU = math.pi * 2

# eslint-disable space-infix-ops

# Calculate an angle between two unit vectors
#
# Since we measure angle between radii of circular arcs,
# we can use simplified math (without length normalization)
#
def unit_vector_angle(ux, uy, vx, vy):
    if(ux * vy - uy * vx < 0):
        sign = -1
    else:
        sign = 1
        
    dot  = ux * vx + uy * vy

    # Add this to work with arbitrary vectors:
    # dot /= math.sqrt(ux * ux + uy * uy) * math.sqrt(vx * vx + vy * vy)

    # rounding errors, e.g. -1.0000000000000002 can screw up this
    if (dot >  1.0): 
        dot =  1.0
        
    if (dot < -1.0):
        dot = -1.0

    return sign * math.acos(dot)


# Convert from endpoint to center parameterization,
# see http:#www.w3.org/TR/SVG11/implnote.html#ArcImplementationNotes
#
# Return [cx, cy, theta1, delta_theta]
#
def get_arc_center(x1, y1, x2, y2, fa, fs, rx, ry, sin_phi, cos_phi):
    # Step 1.
    #
    # Moving an ellipse so origin will be the middlepoint between our two
    # points. After that, rotate it to line up ellipse axes with coordinate
    # axes.
    #
    x1p =  cos_phi*(x1-x2)/2 + sin_phi*(y1-y2)/2
    y1p = -sin_phi*(x1-x2)/2 + cos_phi*(y1-y2)/2

    rx_sq  =  rx * rx
    ry_sq  =  ry * ry
    x1p_sq = x1p * x1p
    y1p_sq = y1p * y1p

    # Step 2.
    #
    # Compute coordinates of the centre of this ellipse (cx', cy')
    # in the new coordinate system.
    #
    radicant = (rx_sq * ry_sq) - (rx_sq * y1p_sq) - (ry_sq * x1p_sq)

    if (radicant < 0):
        # due to rounding errors it might be e.g. -1.3877787807814457e-17
        radicant = 0

    radicant /=   (rx_sq * y1p_sq) + (ry_sq * x1p_sq)
    factor = 1
    if(fa == fs):# Migration Note: note ===
        factor = -1
    radicant = math.sqrt(radicant) * factor #(fa === fs ? -1 : 1)

    cxp = radicant *  rx/ry * y1p
    cyp = radicant * -ry/rx * x1p

    # Step 3.
    #
    # Transform back to get centre coordinates (cx, cy) in the original
    # coordinate system.
    #
    cx = cos_phi*cxp - sin_phi*cyp + (x1+x2)/2
    cy = sin_phi*cxp + cos_phi*cyp + (y1+y2)/2

    # Step 4.
    #
    # Compute angles (theta1, delta_theta).
    #
    v1x =  (x1p - cxp) / rx
    v1y =  (y1p - cyp) / ry
    v2x = (-x1p - cxp) / rx
    v2y = (-y1p - cyp) / ry

    theta1 = unit_vector_angle(1, 0, v1x, v1y)
    delta_theta = unit_vector_angle(v1x, v1y, v2x, v2y)

    if (fs == 0 and delta_theta > 0):#Migration Note: note ===
        delta_theta -= TAU
    
    if (fs == 1 and delta_theta < 0):#Migration Note: note ===
        delta_theta += TAU    

    return [ cx, cy, theta1, delta_theta ]

#
# Approximate one unit arc segment with bezier curves,
# see http:#math.stackexchange.com/questions/873224
#
def approximate_unit_arc(theta1, delta_theta):
    alpha = 4.0/3 * math.tan(delta_theta/4)

    x1 = math.cos(theta1)
    y1 = math.sin(theta1)
    x2 = math.cos(theta1 + delta_theta)
    y2 = math.sin(theta1 + delta_theta)

    return [ x1, y1, x1 - y1*alpha, y1 + x1*alpha, x2 + y2*alpha, y2 - x2*alpha, x2, y2 ]

def a2c(x1, y1, x2, y2, fa, fs, rx, ry, phi):
    sin_phi = math.sin(phi * TAU / 360)
    cos_phi = math.cos(phi * TAU / 360)

    # Make sure radii are valid
    #
    x1p =  cos_phi*(x1-x2)/2 + sin_phi*(y1-y2)/2
    y1p = -sin_phi*(x1-x2)/2 + cos_phi*(y1-y2)/2

    if (x1p == 0 and y1p == 0): # Migration Note: note ===
        # we're asked to draw line to itself
        return []

    if (rx == 0 or ry == 0): # Migration Note: note ===
        # one of the radii is zero
        return []

    # Compensate out-of-range radii
    #
    rx = abs(rx)
    ry = abs(ry)

    lmbd = (x1p * x1p) / (rx * rx) + (y1p * y1p) / (ry * ry)
    if (lmbd > 1):
        rx *= math.sqrt(lmbd)
        ry *= math.sqrt(lmbd)


    # Get center parameters (cx, cy, theta1, delta_theta)
    #
    cc = get_arc_center(x1, y1, x2, y2, fa, fs, rx, ry, sin_phi, cos_phi)

    result = []
    theta1 = cc[2]
    delta_theta = cc[3]

    # Split an arc to multiple segments, so each segment
    # will be less than 90
    #
    segments = int(max(math.ceil(abs(delta_theta) / (TAU / 4)), 1))
    delta_theta /= segments

    for i in range(0, segments):
        result.append(approximate_unit_arc(theta1, delta_theta))

        theta1 += delta_theta
        
    # We have a bezier approximation of a unit circle,
    # now need to transform back to the original ellipse
    #
    return getMappedList(result, rx, ry, sin_phi, cos_phi, cc)

def getMappedList(result, rx, ry, sin_phi, cos_phi, cc):
    mappedList = []
    for elem in result:
        curve = []
        for i in range(0, len(elem), 2):
            x = elem[i + 0]
            y = elem[i + 1]

            # scale
            x *= rx
            y *= ry

            # rotate
            xp = cos_phi*x - sin_phi*y
            yp = sin_phi*x + cos_phi*y

            # translate
            elem[i + 0] = xp + cc[0]
            elem[i + 1] = yp + cc[1]        
            curve.append(complex(elem[i + 0], elem[i + 1]))
        mappedList.append(curve)
    return mappedList


## **Functions**
The findSVGFiles() function is responsible to read all the svg files of the directory. With this function it can
read all the subfolders of the direcory and find the .svg files. It stores directory path of the file along with the file name
in the files_list and the 'category + "//" + filename' in the file_names list.

The create_directory() function is respnsible to create a folder at the given path.

The findTheDPathsOfSVGs() function, is the function that parse the svg files and select the path of the svgs. The svg path along with
the file 'category + "\\" + name' is stored as a tuple.

The bezierCurveAndLineCheck() function takes as input a set of allowed letters to check for the path. The allowed letters from the 
Matlab code I have are {M, C, L, Z} so the purpose of this function is to check if the paths contain a subset of this letters. It is a check
to ensure that the paths have the correct form and discard the ones that does not fullfill the requirement.


In [57]:
from svgpathtools import wsvg, Line, CubicBezier, Path, parse_path
from xml.dom import minidom
import svgpathtools

def findSVGFiles(folder_src_path):
    files_list = []
    file_names = []
    for src_path, src_names, filenames in os.walk(folder_src_path):
        for filename in [f for f in filenames if f.endswith('.svg')]:
            files_list.append(src_path + "\\" + filename)
            #category + name
            file_names.append(src_path.replace(folder_src_path + "\\", "") + "\\" + filename)
    return files_list, file_names

def findTheDPathsOfSVGs(files_list, file_names):
    name_and_path_list = []
    i = 0
    for svg_file in files_list:
        doc = minidom.parse(svg_file)  # parseString also exists
        path_strings = [path.getAttribute('d') for path
                        in doc.getElementsByTagName('path')]
        # The svg files have only one path in my case, that is why i am taking
        # only path_strings[0]
        name_and_path_list.append((file_names[i], path_strings[0]))
        i = i + 1
        doc.unlink()
    return name_and_path_list

def create_directory(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

def bezierCurveAndLineCheck(allowed_letters, name_and_path_list):
    allowed_paths = []
    for i in range(0, len(name_and_path_list)):
        letters = set()
        path_string =  ''.join(filter(str.isalpha, name_and_path_list[i][1]))
        for letter in path_string:
            letters.add(letter)

        # (A Union B)
        a_or_b = allowed_letters.union(letters)
        if len(a_or_b) == len(allowed_letters):
            allowed_paths.append(name_and_path_list[i])
    return allowed_paths

The following code creates the list with the directory paths of the original svgs, and stores the name and the svg path into the name_and_path_list.

In [6]:
icons_source_path = "C:\\Users\\arist\\Desktop\\icon"

files_list, file_names = findSVGFiles(icons_source_path)

name_and_path_list = findTheDPathsOfSVGs(files_list, file_names)


The following code finds the svg files which have arcs, along with curves and lines and not quandratic bezier. The files which have quandratic bezier are only 6 so we discard them from the final dataset. The files which have arcs will be translate into cubic bezier. If they include lines or cubic bezier along with arcs is still fine. Then to take the absolute paths we write the svg files to a folder, so to read the folder with the svgs again later and have the absolute paths.

In [18]:
# find all the paths which contain arcs and not quandratic bezier
path_with_arc_list = []
for i in range(0, len(name_and_path_list)):
    path_alt = parse_path(name_and_path_list[i][1])
    contains_arc = False
    for path in path_alt:
        if path.__class__.__name__ =="Arc":
            contains_arc = True
        elif path.__class__.__name__ =="QuadraticBezier":
            contains_arc = False
            break
    if contains_arc:
        path_with_arc_list.append((name_and_path_list[i][0].replace(icons_source_path, ""), path_alt))

print(len(path_with_arc_list))

# Create new svg with A, C and L
dst_path = "C:\\Users\\arist\\Desktop\\IconsWithArcs\\ReformatedIconsArc\\"
create_directory(dst_path)
for cat_name, path in path_with_arc_list:
    category = cat_name.split("\\")[0]
    create_directory(dst_path + category)
    wsvg(path, filename=dst_path + cat_name)

337
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\ failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\accessibility failed
Successfully created the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\animals 
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc\animals failed
Creation of 

The createCommandDictionary() function is responsible to create the command A, C, L, M for the svg file. The A command has 7 different parameters which are explained very well in this link: https://developer.mozilla.org/en-US/docs/Web/SVG/Tutorial/Paths

The findArcCommand() function is responsible to find the corresponding arc command of the list of dictionaries from the path list of objects of svgpathtools library.

The whole code translates the Arc objects to Cubic Bezier objects and writhe the path (list of Arc, CubicBezier and Line objects) into svg file.

In [54]:
def createCommandDictionary(splitted_path):
    # create a list of dictionaries, each dictionary is a command
    command_list = []
    count = 0
    while count < len(splitted_path):
        command_dict = {}
        if splitted_path[count] == "M" or splitted_path[count] == "L":
            command_dict["name"] = splitted_path[count]
            command_dict["start"] = splitted_path[count+1]
            count = count + 2
            command_list.append(command_dict)
        elif splitted_path[count] == "C":
            command_dict["name"] = splitted_path[count]
            command_dict["start"] = splitted_path[count+1]
            command_dict["controls"] = splitted_path[count+2]
            command_dict["end"] = splitted_path[count+3]
            count = count + 4
            command_list.append(command_dict)
        elif splitted_path[count] == "A":
            command_dict["name"] = splitted_path[count]
            rx_ry = splitted_path[count+1].split(",")
            command_dict["rx"] = rx_ry[0]
            command_dict["ry"] = rx_ry[1]
            command_dict["phi"] = splitted_path[count+2]
            fa_fs = splitted_path[count+3].split(",")
            command_dict["fa"] = fa_fs[0]
            command_dict["fs"] = fa_fs[1]
            x2_y2 = splitted_path[count+4].split(",")
            command_dict["x2"] = x2_y2[0]
            command_dict["y2"] = x2_y2[1]
            if command_dict_prev["name"] == "C":
                x1_y1 = command_dict_prev["end"].split(",")
            elif command_dict_prev["name"] == "L" or command_dict_prev["name"] == "M":
                x1_y1 = command_dict_prev["start"].split(",")
            else:
                x1_y1 = [command_dict_prev["x2"], command_dict_prev["y2"]]
            command_dict["x1"] = x1_y1[0]
            command_dict["y1"] = x1_y1[1]
            count = count + 5
            command_list.append(command_dict)
        else:
            print("Command format is wrong")
        command_dict_prev = command_dict
    return command_list

def findArcCommand(command_list, counter):
    i = 0
    for command in command_list:
        if command["name"] == "A" and counter == i:
            x1 = float(command["x1"])
            y1 = float(command["y1"])
            x2 = float(command["x2"])
            y2 = float(command["y2"])
            fa = float(command["fa"]) 
            fs = float(command["fs"])
            rx = float(command["rx"])
            ry = float(command["ry"])
            phi = float(command["phi"])
            return x1, y1, x2, y2, fa, fs, rx, ry, phi 
        if command["name"] == "A":
            i = i + 1
reformated_list, reformated_names = findSVGFiles(dst_path)

reformated_name_and_path_list = findTheDPathsOfSVGs(reformated_list, reformated_names)

dst_path2 = "C:\\Users\\arist\\Desktop\\IconsWithArcs\\ReformatedIconsArc2\\"
create_directory(dst_path2)
for i in range(0, len(reformated_name_and_path_list)):
    # split the path with space
    svg_path = reformated_name_and_path_list[i][1]
    splitted_path = svg_path.split(" ")
    path_alt = parse_path(svg_path)

    command_list = createCommandDictionary(splitted_path)

    counter = 0
    command_index = 0
    for command_obj in path_alt:
        if command_obj.__class__.__name__ == "Arc":
            path_alt.remove(command_obj)
            x1, y1, x2, y2, fa, fs, rx, ry, phi = findArcCommand(command_list, counter)
            cCommands = a2c(x1, y1, x2, y2, fa, fs, rx, ry, phi)
            for cCommand in cCommands:
                cbezier = CubicBezier(cCommand[0], cCommand[1], cCommand[2], cCommand[3])
                path_alt.insert(command_index, cbezier)
        
            counter = counter + 1
        
        command_index = command_index + 1

    cat_name, path = reformated_name_and_path_list[i]
    cat_name = cat_name.replace(dst_path, "")
    category = category.split("\\")[0]
    create_directory(dst_path2 + category)
    wsvg(path_alt, filename=dst_path2 + cat_name)

Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\ failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIconsArc2\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\ReformatedIcon

In [55]:
# The path that the SVGs are created
reformated_src_path = "C:\\Users\\arist\\Desktop\\IconsWithArcs\\ReformatedIconsArc2"
reformated_list, reformated_names = findSVGFiles(reformated_src_path)

reformated_name_and_path_list2 = findTheDPathsOfSVGs(reformated_list, reformated_names)

A simple check if the reformat of the files contains only the allowed letters.

In [58]:
# the letter e is allowed because it is the power of 10 for some points
allowed_letters = {'M', 'C', 'L', 'e'}

reformated_allowed_paths = bezierCurveAndLineCheck(allowed_letters, reformated_name_and_path_list2)

print(len(reformated_allowed_paths))

337


The function createCommandDictionary() creates a list of dictionaries. Each dictionary is either M, L and C command. If the command is M or L we store their names as "name" and their start coordinates as "start". If it is a C command we store the "name", start coordinates "start", controls "controls" and end coordinates as "end". The coordinates and controls are stored as one string value separated with comma.

The createCcommand() function is responsible to create a sequence of C commands if they are succussive. It returns the sequence string, the end coordinates of the last C command and the index in the list of the command after the last C command.

The createSVGpathList() function creates the path strings depending on the commands. The Matlab code for the stroke removal and deformation has a specific format. If the C commands are successive are allowed to be in the same path. L commands should be separated into different paths, no matter what. When we seperate the commands in different paths the end point of the last command of the previous path is the (M)oving point of the next path. Each path has an M command, and either it has a sequence of C commands (or only one C command) or exactly one L command. 
This code implements the paths following the constraints above.

The createSVGfileFormat() creates an svg file following the exact format of the svg files of the Matlab code.

Running the code we get the final format of the svg files. With this format we are able to run the Matlab code which supports only M, C, L commands.

Matlab Code:https://github.com/yuqian1023/sketch-specific-data-augmentation 

In [61]:
def createCommandDictionary(splitted_path):
    # create a list of dictionaries, each dictionary is a command
    command_list = []
    count = 0
    while count < len(splitted_path):
        command_dict = {}
        if splitted_path[count] == "M" or splitted_path[count] == "L":
            command_dict["name"] = splitted_path[count]
            command_dict["start"] = splitted_path[count+1]
            count = count + 2
            command_list.append(command_dict)
        elif splitted_path[count] == "C":
            command_dict["name"] = splitted_path[count]
            command_dict["start"] = splitted_path[count+1]
            command_dict["controls"] = splitted_path[count+2]
            command_dict["end"] = splitted_path[count+3]
            count = count + 4
            command_list.append(command_dict)
        else:
            print("Command format is wrong")
    return command_list

def createCcommand(command_list, start_index):
    c_string = command_list[start_index]["name"] + " " 
    c_string = c_string + command_list[start_index]["start"] + " "
    c_string = c_string + command_list[start_index]["controls"] + " "
    c_string = c_string + command_list[start_index]["end"]
    end_point = command_list[start_index]["end"]
    end_index = start_index + 1

    for i in range(start_index, len(command_list)):
        if i + 1 > len(command_list) - 1:
            break
        elif command_list[i]["name"] == "C" and command_list[i + 1]["name"] == "C":
            c_string = c_string + " " + command_list[i + 1]["name"] + " " 
            c_string = c_string + command_list[i + 1]["start"] + " "
            c_string = c_string + command_list[i + 1]["controls"] + " "
            c_string = c_string + command_list[i + 1]["end"]
            end_point = command_list[i + 1]["end"]
            end_index = end_index + 1
        else:
            break
    return c_string, end_index, end_point

def createSVGpathList(command_list):
    index = 0
    svg_path_list = []
    while(index < len(command_list)):
        if command_list[index]["name"] == "M":
            m_string = command_list[index]["name"] + " " + command_list[index]["start"]
            if command_list[index + 1]["name"] == "L":
                m_string = m_string + " " + command_list[index + 1]["name"] + " " + command_list[index + 1]["start"]
                global_end_point = command_list[index + 1]["start"]
                index = index + 2
                svg_path_list.append(m_string)
            elif command_list[index + 1]["name"] == "C":
                c_string, end_index, c_end_point = createCcommand(command_list, index + 1)
                index = end_index
                svg_path_list.append(m_string + " " + c_string)
                global_end_point = c_end_point
            else:
                print("Something went wrong")
        elif command_list[index]["name"] == "C":
            c_string, end_index, c_end_point = createCcommand(command_list, index)
            svg_path_list.append("M " + global_end_point + " " + c_string)
            global_end_point = c_end_point
            index = end_index
        elif command_list[index]["name"] == "L":
            l_string = "M " + global_end_point + " " + command_list[index]["name"] + " " + command_list[index]["start"]
            global_end_point = command_list[index]["start"]
            index = index + 1
            svg_path_list.append(l_string)
        else:
            print("Something went wrong")
    return svg_path_list

def createSVGfileFormat(svg_path_list, svg_file_name, stroke_width):
    headline1 = '<?xml version="1.0" encoding="utf-8"?>\n'
    headline2 = '<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n'
    headline3 = '<svg viewBox="0 0 800 800" preserveAspectRatio="xMinYMin meet" xmlns="http://www.w3.org/2000/svg" version="1.1"\n>'
    headline4 = '<g fill="none" stroke="black" stroke-linecap="round" stroke-linejoin="round" stroke-width="' + str(stroke_width) + '">\n'
    headline5 = '<g transform="translate(100,100) scale(1) translate(0,0)">\n'

    with open(svg_file_name, "w+") as f:
        f.write(headline1)
        f.write(headline2)
        f.write(headline3)
        f.write(headline4)
        f.write(headline5)

        count = 0
        for path in svg_path_list:
            path = "<path id=" + '"' + str(count) +'"' + ' d="' + path + '"/>\n'
            count = count + 1
            f.write(path)
        
        f.write('</g>\n')
        f.write('</g>\n')
        f.write('</svg>')
    f.close()

final_format_dst_path = "C:\\Users\\arist\\Desktop\\IconsWithArcs\\FinalFormatIcons\\"
create_directory(final_format_dst_path)

stroke_width = 1.2    
for i in range(0, len(reformated_allowed_paths)):
    # split the path with space
    svg_path = reformated_allowed_paths[i][1]
    splitted_path = svg_path.split(" ")

    command_list = createCommandDictionary(splitted_path)
    svg_path_list = createSVGpathList(command_list)

    category = reformated_allowed_paths[i][0].split("\\")[0]
    create_directory(final_format_dst_path + category)
    createSVGfileFormat(svg_path_list, final_format_dst_path + reformated_allowed_paths[i][0], stroke_width)



Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\ failed
Successfully created the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\accessibility 
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\accessibility failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\accessibility failed
Successfully created the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\animals 
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\animals failed
Creation of the directory C:\Users\arist\Desktop\IconsWithArcs\FinalFormatIcons\animals failed
Creation of the directory C:\User